In [1]:
import os,glob
import numpy as np
import pandas as pd
import glob
import fastjet
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os
from atlas_topq_2019_23_Recast import getLHEevents

pd.option_context('display.max_columns', -1)

pd.options.mode.chained_assignment = None #Disable copy warnings
# plt.style.use('fivethirtyeight') #Set style
# mpl.rcParams.update({'figure.figsize' : (15,10)})  #Set general plotting options
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.colormaps['RdYlBu']

In [2]:
BR = 2*(0.6741*(0.1071+0.1063)) # Used to rescale the MG5 cross-section if the decay is included in the process
f = '../../processFolders/sm_ttdec_lo/Events/run_01/unweighted_events.lhe.gz'

In [3]:
etamax=2.0
pTmin=355.0

atlas_bins = np.array([355.0,381.0,420.0,478.0,549.0,633.0,720.0,836.0,2000.0])
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 0.4)
fatjetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.0)

totalweightPB = 0.0
pTT = []
weights = []
cutFlow = {'n(l) = 1' : np.zeros(2), 
           'n(j) > 0' : np.zeros(2), 
           'n(FJ) > 0' : np.zeros(2), 
           'pT(FT) > %1.0f' %pTmin : np.zeros(2), 
           'eta(FT) < %1.1f' %etamax : np.zeros(2), 
           'b-jet in FT' : np.zeros(2),  
           'pT(lep) > 27' : np.zeros(2), 
           'eta(lep) < 2.5' : np.zeros(2), 
           'dR(lepton,b) < 2.0' : np.zeros(2), 
           'm(lepton,b) < 180' : np.zeros(2), 
           'dR(lepton,j) > 0.4' : np.zeros(2),
           'MET > 20' : np.zeros(2)}

In [4]:
ntotal,events = getLHEevents(f)


for ev in events:
    error = False
    weightPB = ev.eventinfo.weight/ntotal
    weightAndError = np.array([weightPB,weightPB**2])
    totalweightPB += weightPB

    # Add information to particles:
    for ptc in ev.particles:
        ptc.daughters = []
        if ptc.status < 0: # Skip incoming partons
            continue
        p = np.sqrt(ptc.px**2 + ptc.py**2 + ptc.pz**2)
        ptc.Eta = (1./2.)*np.log((p+ptc.pz)/(p-ptc.pz))
        ptc.PT = np.sqrt(ptc.px**2 + ptc.py**2)
        ptc.Phi = np.arctan2(ptc.py,ptc.px)
        ptc.Px = ptc.px
        ptc.Py = ptc.py
        ptc.Pz = ptc.pz
        ptc.E = ptc.e
        ptc.PID = int(ptc.id)
    for ptc in ev.particles:
        for mom in ptc.mothers():
            mom.daughters.append(ptc)
    
    # Get tops and their final decays:
    tops = {}
    topDecays = {}
    for ptc in ev.particles:        
        if abs(ptc.PID) == 6:
            tops[ptc.PID] = ptc # Store only the last top/anti-top            
            daughters = {d.PID : d for d in ptc.daughters}
            hasDaughters = [pid for pid,d in daughters.items() if d.daughters]
            while len(hasDaughters) > 0:
                for pid in hasDaughters:
                    d = daughters.pop(pid)
                    for dd in d.daughters:
                        daughters[dd.PID] = dd
                hasDaughters = [pid for pid,d in daughters.items() if d.daughters]
            topDecays[ptc.PID] = list(daughters.values())
    
    # Select events with one lepton and one hadronic top:
    topH = None
    topLep = None
    for topPID,daughters in topDecays.items():        
        dPIDs =  [abs(ptc.PID) for ptc in daughters]
        if len(dPIDs) != 3:
            error = True
            print('Error getting daughters:',dPIDs)
            break
        if not 5 in dPIDs:
            continue # Skip rare decays to W+c
        if (11 in dPIDs) or (13 in dPIDs):
            topLep = topPID
        elif max(dPIDs) <= 5:
            topH = topPID

    if error:
        break
    
    if topH is None or topLep is None:
        continue
   
    cutFlow['n(l) = 1'] += weightAndError

    # Hadronic top:    
    # Regular jets:
    quarks = [ptc for ptc in topDecays[topH]]
    jetArray = [fastjet.PseudoJet(q.Px,q.Py,q.Pz,q.E) for q in quarks if abs(q.Eta) < 4.5]
    for ij,j in enumerate(jetArray):
        j.set_user_index(quarks[ij].PID)
    cluster = fastjet.ClusterSequence(jetArray, jetdef)
    jets = cluster.inclusive_jets(ptmin = 26.0)
    jets = [j for j in jets if abs(j.eta()) < 2.5]
    if len(jets) == 0:
        continue
    cutFlow['n(j) > 0'] += weightAndError

        
    # ## Fat jet:    
    jetArray = [fastjet.PseudoJet(j.px(),j.py(),j.pz(),j.E()) for j in jets]
    for ij,j in enumerate(jetArray):
        for q in jets[ij].constituents():
            if abs(q.user_index()) == 5:
                j.set_user_index(5) # Tag the regular jets containing a b-quark
    clusterFat = fastjet.ClusterSequence(jetArray, fatjetdef)
    if len(clusterFat.inclusive_jets()) == 0:
        continue
    cutFlow['n(FJ) > 0'] += weightAndError

    # Use hardest fat jet
    fatJet = sorted([j for j in clusterFat.inclusive_jets()], key = lambda j: j.pt(), reverse=True)[0]
    # Invariant mass cut:
    if not (120. < fatJet.m() < 220.):
        continue
    # PT cut
    if fatJet.pt() < pTmin:
        continue
    cutFlow['pT(FT) > %1.0f' %pTmin] += weightAndError

    # Eta cut
    if abs(fatJet.eta()) > etamax:
        continue
    cutFlow['eta(FT) < %1.1f' %etamax] += weightAndError

    # Require a b inside the Fat jet
    hasB = False
    for q in fatJet.constituents():
        if q.user_index() == 5:
            hasB = True
    if not hasB:
        continue
    cutFlow['b-jet in FT'] += weightAndError

        
    # Leptonic top:
    leptons = [ptc for ptc in topDecays[topLep] if abs(ptc.PID) in [11,13]]
    neutrinos = [ptc for ptc in topDecays[topLep] if abs(ptc.PID) in [12,14]]
    bLep = [ptc for ptc in topDecays[topLep] if abs(ptc.PID) ==5]
    if len(leptons) != 1:
        error = True
        print('Error getting leptons')
    if len(neutrinos) != 1:
        error = True
        print('Error getting neutrinos')
    if len(bLep) != 1:
        error = True
        print('Error getting b-jet')
        break
    if error:
        break
    lepton = leptons[0]
    nu = neutrinos[0]
    bLep = bLep[0]
    
    pTlepton = lepton.PT
    etaLep = np.abs(lepton.Eta)
    # Lepton pT cut
    if pTlepton < 27.0:
        continue
    cutFlow['pT(lep) > 27'] += weightAndError

    # Lepton eta cut
    if etaLep > 2.5:
        continue
    cutFlow['eta(lep) < 2.5'] += weightAndError

    # Require the b to be close to the lep'MET > 20'ton
    dRlep = np.sqrt((lepton.Eta-bLep.Eta)**2 + (lepton.Phi-bLep.Phi)**2)
    if dRlep > 2.0:
        continue
    cutFlow['dR(lepton,b) < 2.0'] += weightAndError

    # Invariant mass of lepton and b < 180:
    mlb = np.sqrt((lepton.E+bLep.E)**2-(lepton.Px+bLep.Px)**2-(lepton.Py+bLep.Py)**2-(lepton.Pz+bLep.Pz)**2)
    if mlb > 180.0:
        continue
    cutFlow['m(lepton,b) < 180'] += weightAndError

    # Skip events where lepton overlaps to jet
    dRlep = min([np.sqrt((lepton.Eta-j.eta())**2 + (lepton.Phi-j.phi())**2) for j in jets])    
    if dRlep < 0.4:
        continue
    cutFlow['dR(lepton,j) > 0.4'] += weightAndError

    # MET cut
    if (nu.PT < 20.0):
        continue
    cutFlow['MET > 20'] += weightAndError

    pTT.append(tops[topH].PT)
    weights.append(weightAndError)
    


weights = np.array(weights)
pTtHist,_ = np.histogram(pTT,weights=weights[:,0],bins=atlas_bins)
pTtHistError,_ = np.histogram(pTT,weights=weights[:,1],bins=atlas_bins)
pTtHistError = np.sqrt(pTtHistError)
for val in cutFlow.values():
    val[1] = np.sqrt(val[1])

data = np.array(list(zip(atlas_bins[:-1],atlas_bins[1:],pTtHist,pTtHistError)))
    

In [ ]:
# Inclusive cross-section (total ttbar production)
inclusiveXsec = totalweightPB/BR
# Compute k-factor based on the inclusive production cross-section:
kfac = 832./inclusiveXsec

print('xsec (pb) = %1.3e' %(inclusiveXsec))
print('fiducial xsec*kfac (pb) = %1.3e' %(sum(weights[:,0])*kfac))


In [ ]:
xsec_before = totalweightPB
for c,xsec in cutFlow.items():
    print('%s : %1.2f +- %1.0g (%1.2f%%)' %(c,xsec[0]*kfac,xsec[1]*kfac,100*xsec[0]/xsec_before))
    xsec_before = xsec[0]

In [ ]:
sm_nnlo = np.loadtxt('./digitised_sm/nnlo_from_fig11.txt',dtype=float,usecols=(0,))
bins_width = atlas_bins[1:]-atlas_bins[:-1]

In [ ]:
for ibin,pt in enumerate(data):
    r = pt[2]*kfac/(sm_nnlo[ibin]*bins_width[ibin])
    rErr = pt[3]*kfac/(sm_nnlo[ibin]*bins_width[ibin])
    print('Bin: %1.0f--%1.0f' %(pt[0],pt[1]))
    print("MG5 = %1.2e, ATLAS = %1.2e (ratio = %1.2f +- %1.1f)\n" %(pt[2]*kfac,sm_nnlo[ibin]*bins_width[ibin],r,rErr))

In [ ]:
plt.hist(pTT,weights=weights[:,0],bins=atlas_bins,histtype='step')
plt.errorbar((atlas_bins[:-1]+atlas_bins[1:])/2, pTtHist, yerr=pTtHistError, fmt='o')
plt.yscale('log')
# plt.xlim(500.,1500.)
# plt.ylim(1e-3,1e-1)
plt.show()

In [ ]:
max(weights[:,0]),min(weights[:,0])

In [ ]:
2.87e-02/max(weights[:,0])

In [ ]:
1./np.sqrt(54.0)

In [ ]:
ntotal

In [ ]:
ptc.__dict__